In [ ]:
import petl as etl

from utils.paths import PUBLISHED, SITE

import json

In [ ]:
responses = (
    etl.fromcsv(PUBLISHED / 'surveys/responses.csv')
    .selectne('month', '')
    .convertnumbers()
    .addfield('aggregate_nps', lambda r: r.nps * r.responses)
    .cache()
)

In [ ]:
total = (
    responses
    .aggregate(None, {
        'responses': ('responses', sum),
        'rated_good_or_excellent': ('rated_good_or_excellent', sum),
        'nps': ('aggregate_nps', sum),
    })
    .convert('nps', lambda f, r: f / r.responses, pass_row=True)
    .addfield('proportion', lambda r: r.rated_good_or_excellent / r.responses)
)



In [ ]:
by_month = (
    responses
    .aggregate('month', {
        'responses': ('responses', sum),
        'rated_good_or_excellent': ('rated_good_or_excellent', sum),
        'nps': ('aggregate_nps', sum),
    })
    .convert('nps', lambda f, r: f / r.responses, pass_row=True)
    .addfield('proportion', lambda r: r.rated_good_or_excellent / r.responses)
    .sort('month')
)

In [ ]:
OUT = SITE / '_data/audience_survey'
OUT.mkdir(exist_ok=True, parents=True)

with open(OUT / 'total.json', 'w') as f:
    json.dump(dict(total.transpose()), f, indent=2)

by_month.tojson(OUT / 'by_month.json', indent=2)